In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as     np
from   numba import njit
from src_dir import Gauss_pdf_2D, GMRES, GMRES_numba, mk_laplace_2d, laplace_2d_dirichlet

In [25]:
# Set dimension of the NxN grid used
# Note: For optimal performance, the neural network "cnn_collectionOnline2D.py"
# can be tweaked with appropriate kernel dilations, however the code should
# still work and yield resuluts for any dimension of input provided
dim = 80

# Default initial guess used for direct un-preconditioned GMRES is the zero
# solution
x0 = np.squeeze(np.zeros((dim,dim)))
x0Type = 'Zero Solution 2D'

# Set tolerances for GMRES solver
e = 1e-10

# Restarted GMRES parameters
nmax_iter = 10
restart   = 1000

# Create domain [-1,1]x[-1,1]
# Define grid values at midpoints of cartesian grid
DomainL = -1.0
DomainR =  1.0
dx = (DomainR-DomainL)/(dim-1)
x1 = np.linspace(DomainL+dx,DomainR-dx,dim)
x2 = np.linspace(DomainL+dx,DomainR-dx,dim)
X, Y = np.meshgrid(x1, x2, sparse=False, indexing='ij')
Area = (dx*(dim-1))**2

# Create 2D laplace opertor as a stencil opertor for a N-cell 2D grid
# Can be found in linop.py in src_di
A = mk_laplace_2d(dim, dim, bc="dirichlet", xlo=0, xhi=0, ylo=0, yhi=0)
AType = '2D Laplacian'

# Total number of steps in simulation
n_steps = 100

In [26]:
def particle(X, Y, x, y, charge, sigma):
    b = charge*Gauss_pdf_2D(X, Y, x, y, sigma)
    return b

def force(dx, X, Y, phi_x, phi_y, x, y,sigma):
    weights = Gauss_pdf_2D(X, Y, x, y, sigma)
    return np.array([
        sum(phi_x*weights),
        sum(phi_y*weights)
    ])*dx**2

In [27]:
p_0    = np.array([[0, 0], [0.1, -0.5], [0.2, 0.5], [-0.1, 0.5], [-0.2, -0.5]])
charge = np.array([ 1,      1,           -1,         -1,          1])

In [28]:
b = particle(X, Y, *p_0[0, :], charge[0], 0.05)
for i in range(1, len(charge)):
    b += particle(X, Y, *p_0[i, :], charge[i], 0.05)

In [29]:
@njit(nogil=True)
def A_numba(x):
    return laplace_2d_dirichlet(x, 0, 0, 0, 0)

In [32]:
%time _ = A(b)

CPU times: user 73.4 ms, sys: 4.34 ms, total: 77.8 ms
Wall time: 79.7 ms


In [33]:
%time _ = A_numba(b)

CPU times: user 104 µs, sys: 11 µs, total: 115 µs
Wall time: 119 µs


In [34]:
%time _ = GMRES(A, b, x0, e, nmax_iter, restart, True)

CPU times: user 3min 15s, sys: 7.68 s, total: 3min 22s
Wall time: 3min 29s


In [35]:
%time _ = GMRES(A_numba, b, x0, e, nmax_iter, restart, True)

CPU times: user 1.05 s, sys: 145 ms, total: 1.2 s
Wall time: 1.22 s


In [47]:
%time _ = GMRES_numba(A_numba, b, x0, e, nmax_iter, restart, True)

CPU times: user 517 ms, sys: 103 ms, total: 619 ms
Wall time: 629 ms
